In [ ]:
import glob
from skimage.io import imread

path = "data"
train_names = glob.glob(path + "/trainResized/*")
train_files = []
for nameFile in train_names:
    train_files.append(imread(nameFile))

test_names = glob.glob(path + "/testResized/*")
test_files = []
for nameFile in test_names:
    test_files.append(imread(nameFile))

print(train_files[0].shape, train_files[0].type)
